In [ ]:
import mirdata
import os
import librosa
import numpy as np
import IPython.display as ipd
import pandas as pd
from scipy.io import wavfile

In [ ]:
audio_path = os.path.join('svara_task', 'svara_task', 'Kamakshi', 'Sanjay Subrahmanyan - Kamakshi.mp3')

y, sr = librosa.load(audio_path, sr=None)

In [ ]:
print('y:', y)
print (y.shape)

In [ ]:
# display audio
ipd.Audio(y, rate=sr)



In [ ]:
svara_path = os.path.join('svara_task', 'svara_task', 'kamakshi_new.txt')

# load the content of svara_path.txt as a dataframe with pandas
headers = ['Level', 'Start', 'End', 'Duration', 'Label']
svara = pd.read_csv(svara_path, sep='\t', names=headers)

print(svara)
print(svara.shape)


svara.insert(0, 'ID', range(1, len(svara) + 1))
svara.set_index('ID', inplace=True)
print(svara)
print(svara.shape)



In [ ]:
# svara_time = pd.DataFrame(columns=headers)

for index, row in svara.iterrows():
    # for a given row, take the start and end time and the svara (columns 2 and 3)

    start_time = row[1]
    end_time = row[2]
    duration = row[3]

    # print('start_time:', start_time)
    # print('end_time:', end_time)

    # convert start and end time from string to float
    hours_s, minutes_s, seconds_s = map(float, start_time.split(':'))
    seconds_s += hours_s * 3600 + minutes_s * 60

    hours_e, minutes_e, seconds_e = map(float, end_time.split(':'))
    seconds_e += hours_e * 3600 + minutes_e * 60

    hours_d, minutes_d, seconds_d = map(float, duration.split(':'))
    seconds_d += hours_d * 3600 + minutes_d * 60

    # Update the current row with the modified values
    svara.at[index, 'Start'] = seconds_s
    svara.at[index, 'End'] = seconds_e
    svara.at[index, 'Duration'] = seconds_d





print(svara)

In [ ]:
# extract the audio segment corresponding to the row
start_sample = int(seconds_s * sr)
end_sample = int(seconds_e * sr)
y_segment = y[start_sample:end_sample]

# display the audio segment
ipd.Audio(y_segment, rate=sr)


In [ ]:
svara_list = ['sa', 'ri', 'ga', 'ma', 'pa', 'dha', 'ni']
# filter dataframe by Label equals 'sa'
for element in svara_list:
    # Filter the original DataFrame based on the current element
    filtered_df = svara[svara['Label'] == element]

    # Assign the filtered DataFrame to a new variable dynamically
    globals()['svara_' + element] = filtered_df

    # Print the filtered DataFrame
    print('svara_' + element + ':')
    print(filtered_df)


In [ ]:
# create a folder named 'single_svara' to store the audio segments
if not os.path.exists('single_svara'):
    os.makedirs('single_svara')

for element in svara_list:

# extract the audio segment corresponding to the row for 'sa'
    for index, row in globals()['svara_' + element].iterrows():
        start_sample = int(row[1] * sr)
        end_sample = int(row[2] * sr)
        y_segment = y[start_sample:end_sample]
        # save the audio segment to a file
        audio_segment_path = os.path.join('single_svara', element +'_' + str(index) + '.wav')
        wavfile.write(audio_segment_path, sr, y_segment)
